# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] GraphLab Create v1.8.3 started. Logging: C:\Users\Vinay\AppData\Local\Temp\graphlab_server_1465660917.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.430650     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.466740     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [7]:
model_all.get('coefficients')

name,index,value,stderr
(intercept),None,274873.05595,None
bedrooms,None,0.0,None
bedrooms_square,None,0.0,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [13]:
def RSS(predicted,data):
    diffn=predicted-data
    Rss=(diffn*diffn).sum()
    return Rss

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [30]:
import numpy as np
error_tracker=[]
count=0

for l in np.logspace(1,7,num=13):
    Error=0
    count=+1
    modelname= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l)
    predictions=modelname.predict(validation)
    Error= RSS(predictions,validation['price'])
    error_tracker.appen(Error)

    print "with penalty of ",l, " we get valid error of ", Error
    
least= error_tracker.index(min(error_tracker))
print least
    
    



with penalty of  10.0  we get valid error of  6.25766285142e+14
with penalty of  31.6227766017  we get valid error of  6.25766285362e+14
with penalty of  100.0  we get valid error of  6.25766286058e+14
with penalty of  316.227766017  we get valid error of  6.25766288257e+14
with penalty of  1000.0  we get valid error of  6.25766295212e+14
with penalty of  3162.27766017  we get valid error of  6.25766317206e+14
with penalty of  10000.0  we get valid error of  6.25766386761e+14
with penalty of  31622.7766017  we get valid error of  6.25766606749e+14
with penalty of  100000.0  we get valid error of  6.25767302792e+14
with penalty of  316227.766017  we get valid error of  6.25769507644e+14
with penalty of  1000000.0  we get valid error of  6.25776517727e+14
with penalty of  3162277.66017  we get valid error of  6.25799062845e+14
with penalty of  10000000.0  we get valid error of  6.25883719085e+14


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [33]:
count=0
error_tracker=[]
for l in np.logspace(1,7,num=13):
    Error=0
    count=+1
    modelname= graphlab.linear_regression.create(testing, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l)
    predictions=modelname.predict(validation)
    Error= RSS(predictions,validation['price'])
    error_tracker.append(Error)

    print "with penalty of ",l, " we get valid error of ", Error

least= error_tracker.index(min(error_tracker))
print least
    

with penalty of  10.0  we get valid error of  6.31133636474e+14
with penalty of  31.6227766017  we get valid error of  6.31133639845e+14
with penalty of  100.0  we get valid error of  6.31133650506e+14
with penalty of  316.227766017  we get valid error of  6.31133684216e+14
with penalty of  1000.0  we get valid error of  6.31133790818e+14
with penalty of  3162.27766017  we get valid error of  6.31134127932e+14
with penalty of  10000.0  we get valid error of  6.31135194054e+14
with penalty of  31622.7766017  we get valid error of  6.31138566194e+14
with penalty of  100000.0  we get valid error of  6.31149237478e+14
with penalty of  316227.766017  we get valid error of  6.31183059452e+14
with penalty of  1000000.0  we get valid error of  6.31290778004e+14
with penalty of  3162277.66017  we get valid error of  6.31639054896e+14
with penalty of  10000000.0  we get valid error of  6.3281681206e+14
0


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [34]:
modelname= graphlab.linear_regression.create(testing, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=10.0)
modelname.get('coefficients')

name,index,value,stderr
(intercept),None,15938.3491302,None
bedrooms,None,7927.96151068,None
bedrooms_square,None,2811.6551531,None
bathrooms,None,25090.812508,None
sqft_living,None,39.6894164796,None
sqft_living_sqrt,None,1115.10438309,None
sqft_lot,None,-0.15538020793,None
sqft_lot_sqrt,None,126.735040031,None
floors,None,19806.9697903,None
floors_square,None,12646.6371612,None


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [35]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [36]:
l1_penalty_values = np.logspace(8, 10, num=20)


count=0
error_tracker=[]
for l in l1_penalty_values:
    Error=0
    count=+1
    modelname= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l)
    predictions=modelname.predict(validation)
    Error= RSS(predictions,validation['price'])
    error_tracker.append(Error)

    print "with penalty of ",l, " we get valid error of ", Error,"and it has following Non zero weights"
    print modelname['coefficients']['value'].nnz()
    






with penalty of  100000000.0  we get valid error of  6.27492659875e+14 and it has following Non zero weights
18
with penalty of  127427498.57  we get valid error of  6.28210516771e+14 and it has following Non zero weights
18
with penalty of  162377673.919  we get valid error of  6.29176689541e+14 and it has following Non zero weights
18
with penalty of  206913808.111  we get valid error of  6.30650082719e+14 and it has following Non zero weights
18
with penalty of  263665089.873  we get valid error of  6.32940229287e+14 and it has following Non zero weights
17
with penalty of  335981828.628  we get valid error of  6.3626814023e+14 and it has following Non zero weights
17
with penalty of  428133239.872  we get valid error of  6.41261198311e+14 and it has following Non zero weights
17
with penalty of  545559478.117  we get valid error of  6.48983455376e+14 and it has following Non zero weights
17
with penalty of  695192796.178  we get valid error of  6.60962217696e+14 and it has followin

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [37]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [38]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [41]:
count=0
error_tracker=[]
for l in l1_penalty_values:
    Error=0
    count=+1
    modelname= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l)
    predictions=modelname.predict(validation)
    Error= RSS(predictions,validation['price'])
    error_tracker.append(Error)

    print "with penalty of ",l, " we get valid error of ", Error,"and it has following Non zero weights"
    print modelname['coefficients']['value'].nnz()

with penalty of  2976351441.63  we get valid error of  9.66925692362e+14 and it has following Non zero weights
10
with penalty of  3019316638.95  we get valid error of  9.74019450085e+14 and it has following Non zero weights
10
with penalty of  3062281836.27  we get valid error of  9.81188367942e+14 and it has following Non zero weights
10
with penalty of  3105247033.59  we get valid error of  9.89328342459e+14 and it has following Non zero weights
10
with penalty of  3148212230.91  we get valid error of  9.98783211266e+14 and it has following Non zero weights
10
with penalty of  3191177428.24  we get valid error of  1.00847716702e+15 and it has following Non zero weights
10
with penalty of  3234142625.56  we get valid error of  1.01829878055e+15 and it has following Non zero weights
10
with penalty of  3277107822.88  we get valid error of  1.02824799221e+15 and it has following Non zero weights
10
with penalty of  3320073020.2  we get valid error of  1.03461690923e+15 and it has follo

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [42]:
modelname= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=3448968612.16)
predictions=modelname.predict(validation)
Error= RSS(predictions,validation['price'])


print "with penalty of ",l, " we get valid error of ", Error,"and it has following Non zero weights"
print modelname['coefficients']['value'].nnz()

with penalty of  3792690190.73  we get valid error of  1.04693748875e+15 and it has following Non zero weights
7
